# Lab 3

---------

CSC3310 - 002

Leena Afifi

Vlad Wilson

## Abstract

In [ ]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

if __name__ == "__main__":
    # Define a point to test
    point = Point(179, 10)

    # Define a polygon
    polygon = [
        Point(186, 14),
        Point(186, 44),
        Point(175, 115),
        Point(175, 85)
    ]

    # Check if the point is inside the polygon
    if point_in_polygon(point, polygon):
        print("Point is inside the polygon")
    else:
        print("Point is outside the polygon")

Point is outside the polygon
